<a href="https://colab.research.google.com/github/wojciyo-maker/librus-extractor/blob/new-features/PDF_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##pip install tabula-py

In [ ]:
import tabula
import pandas as pd

# Path to your PDF file
pdf_path = '/content/Zosia_Oceny.pdf'

# Output CSV file name
output_csv_path = 'Zosia_Oceny.csv'

# Extract tables from the PDF
# pages='all' extracts from all pages
# lattice=True is often helpful for PDFs with well-defined tables
tables = tabula.read_pdf(pdf_path, pages='1', multiple_tables=True, stream=True, lattice=False)

# If multiple tables are extracted, you might need to concatenate them or process individually
# For simplicity, let's assume we are interested in the first table found or concatenate all.
if tables:
    # Concatenate all extracted tables into a single DataFrame
    df = pd.concat(tables)

    # Save the DataFrame to a CSV file
    df.to_csv(output_csv_path, index=False)
    print(f"Successfully extracted data to {output_csv_path}")

    # Display the first 5 rows of the extracted data
    print("\nFirst 5 rows of the extracted data:")
    display(df.head())
else:
    print(f"No tables found in {pdf_path}")

In [ ]:
# Make a copy to avoid modifying the original DataFrame directly yet
df_cleaned = df.copy()

# Drop the first two rows which contain header information that is not useful in the data
df_cleaned = df_cleaned.iloc[2:].reset_index(drop=True)

# Rename 'Unnamed: 0' to 'Przedmiot' early to make subsequent column identification clearer
df_cleaned = df_cleaned.rename(columns={'Unnamed: 0': 'Przedmiot'})

# Identify individual grade columns for 'Oceny bieżące'
oceny_biezace_cols_candidates = [
    'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
    'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'
]

# Filter for existing columns in the DataFrame
oceny_biezace_cols = [col for col in oceny_biezace_cols_candidates if col in df_cleaned.columns]

# Combine individual grade columns into 'Oceny_bieżące'
if oceny_biezace_cols:
    df_cleaned['Oceny_bieżące'] = df_cleaned[oceny_biezace_cols].fillna('').astype(str).agg(' '.join, axis=1)
    df_cleaned['Oceny_bieżące'] = df_cleaned['Oceny_bieżące'].str.strip().str.replace(r'\s+', ' ', regex=True)
else:
    df_cleaned['Oceny_bieżące'] = '' # Create an empty column if no source columns exist

# Ensure 'Okres 1' and 'Okres 2' are present and cleaned (fill NaN with empty string)
for col_name in ['Okres 1', 'Okres 2']:
    if col_name in df_cleaned.columns:
        df_cleaned[col_name] = df_cleaned[col_name].fillna('')
    else:
        df_cleaned[col_name] = '' # Create if missing

# Merge 'Oceny_bieżące' into 'Okres 1'
# If 'Okres 1' is empty, use 'Oceny_bieżące'. If 'Okres 1' has content and 'Oceny_bieżące' also has content, concatenate them.
# If 'Okres 1' is empty and 'Oceny_bieżące' is also empty, it remains empty.
df_cleaned['Okres 1'] = df_cleaned.apply(lambda row:
    (row['Oceny_bieżące'].strip() + ' ' + row['Okres 1'].strip()).strip()
    if row['Oceny_bieżące'].strip() and row['Okres 1'].strip() else
    row['Oceny_bieżące'].strip() if row['Oceny_bieżące'].strip() else
    row['Okres 1'].strip(), axis=1)


# Identify all columns to drop:
# 1. The original individual grade columns that were merged into 'Oceny_bieżące'.
# 2. 'Oceny_bieżące' itself, as it has been merged into 'Okres 1'.
# 3. Any other 'Unnamed:' columns that are not 'Przedmiot'.
# 4. The 'Ocena' column if it was created in a previous incorrect run.
columns_to_drop_final = []
columns_to_drop_final.extend(oceny_biezace_cols) # Original individual grade columns
columns_to_drop_final.append('Oceny_bieżące') # The temporary combined column
columns_to_drop_final.extend([col for col in df_cleaned.columns if 'Unnamed:' in col and col != 'Przedmiot' and col not in columns_to_drop_final])
if 'Ocena' in df_cleaned.columns:
    columns_to_drop_final.append('Ocena') # Drop the previously combined 'Ocena' column

# Remove duplicates and ensure only existing columns are dropped
columns_to_drop_final = list(set(columns_to_drop_final))
columns_to_drop_final = [col for col in columns_to_drop_final if col in df_cleaned.columns]

df_cleaned = df_cleaned.drop(columns=columns_to_drop_final)

# Reorder columns for better presentation
desired_order = ['Przedmiot', 'Okres 1', 'Okres 2', 'Koniec roku']
existing_cols_in_order = [col for col in desired_order if col in df_cleaned.columns]
remaining_cols = [col for col in df_cleaned.columns if col not in desired_order]
df_cleaned = df_cleaned[existing_cols_in_order + remaining_cols]

# Assign the cleaned DataFrame back to df
df = df_cleaned.copy()

# Display the first few rows of the cleaned DataFrame
print("\nCleaned DataFrame head:")
display(df.head(10))